In [3]:
# 필요한 라이브러리 임포트
import pandas as pd  # 데이터 처리를 위한 pandas
import numpy as np  # 수치 계산을 위한 numpy
from sklearn.model_selection import train_test_split, cross_val_score  # 데이터 분할 및 교차 검증
from sklearn.impute import SimpleImputer  # 결측치 처리
from sklearn.preprocessing import OneHotEncoder, StandardScaler  # 범주형 변수 인코딩 및 수치형 변수 스케일링
from sklearn.compose import ColumnTransformer  # 컬럼별 전처리 파이프라인 구성
from sklearn.pipeline import Pipeline  # 전체 전처리 및 모델링 파이프라인 구성
from sklearn.linear_model import Ridge, Lasso, ElasticNet  # 선형 회귀 모델
from sklearn.metrics import mean_squared_error, make_scorer  # 모델 평가 지표
from sklearn.model_selection import RandomizedSearchCV  # 랜덤 서치를 통한 하이퍼파라미터 튜닝
from skopt import BayesSearchCV  # 베이지안 최적화를 통한 하이퍼파라미터 튜닝
import xgboost as xgb  # XGBoost 모델
import lightgbm as lgb  # LightGBM 모델
import warnings  # 경고 메시지 처리
import joblib  # 모델 저장 및 로드
import os  # 파일 시스템 작업
import json  # JSON 파일 처리

# LightGBM의 불필요한 경고 메시지 무시 설정
warnings.filterwarnings('ignore', category=UserWarning, module='lightgbm')

# 데이터 로드
train = pd.read_csv("train.csv")

# 특성 선택
drop_cols = ['id', 'Calories']
numeric_feats = train.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_feats = train.select_dtypes(include=['object']).columns.tolist()

# drop_cols 제외
numeric_feats = [col for col in numeric_feats if col not in drop_cols]
categorical_feats = [col for col in categorical_feats if col not in drop_cols]
main_features = numeric_feats + categorical_feats

print("Numeric features:", numeric_feats)
print("Categorical features:", categorical_feats)
print("Total features:", len(main_features))

y = train['Calories']
X = train[main_features]

# 수치형 변수 전처리 파이프라인 구성
numeric_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),  # 결측치 처리
    ('scaler', StandardScaler())  # 표준화 스케일링 적용
])

# 범주형 변수 전처리 파이프라인 구성
categorical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),  # 결측치 처리
    ('encoder', OneHotEncoder(handle_unknown='ignore'))  # 원-핫 인코딩 적용
])

# 전처리 파이프라인 통합
preprocessor = ColumnTransformer([
    ('num', numeric_transformer, numeric_feats),  # 수치형 데이터 처리
    ('cat', categorical_transformer, categorical_feats),  # 범주형 데이터 처리
])

# 2. 모델 후보군 정의
models = {
    'Ridge': Ridge(),  # 릿지 회귀
    'Lasso': Lasso(),  # 라쏘 회귀
    'ElasticNet': ElasticNet(),  # 엘라스틱넷 회귀
    'XGBoost': xgb.XGBRegressor(tree_method='hist', random_state=42),  # XGBoost
    'LightGBM': lgb.LGBMRegressor(random_state=42, verbose=-1)  # LightGBM
}

results = {}  # 결과 저장 딕셔너리
for name, model in models.items():
    print(f'\n==== {name} ====')
    try:
        # 3.1 전체 파이프라인 구성 (전처리 + 모델)
        pipe = Pipeline([
            ('preprocessor', preprocessor),  # 전처리 단계
            ('reg', model)  # 모델 단계
        ])
        
        # 3.2 모델 학습
        pipe.fit(X, y)
        
        # 3.3 예측 및 성능 평가
        y_pred = pipe.predict(X)  # 예측값 생성
        rmse = np.sqrt(mean_squared_error(y, y_pred))  # RMSE 계산
        print(f'RMSE: {rmse:.4f}')
        
        results[name] = rmse  # 결과 저장
        
    except Exception as e:
        print(f"Error: {str(e)}")

# 4. 최종 결과 비교 및 정렬
print("\n==== Final Results ====")
results_df = pd.DataFrame(results.items(), columns=['Model', 'RMSE'])  # 결과 데이터프레임 생성
results_df = results_df.sort_values('RMSE')  # RMSE 기준 오름차순 정렬
print(results_df)

Numeric features: ['Age', 'Height', 'Weight', 'Duration', 'Heart_Rate', 'Body_Temp']
Categorical features: ['Sex']
Total features: 7

==== Ridge ====
RMSE: 11.0974

==== Lasso ====
RMSE: 12.0979

==== ElasticNet ====
RMSE: 19.4973

==== XGBoost ====
RMSE: 3.5798

==== LightGBM ====


C:\Users\Admin\Desktop\lgu6P_ml_dl\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


RMSE: 3.7663

==== Final Results ====
        Model       RMSE
3     XGBoost   3.579833
4    LightGBM   3.766349
0       Ridge  11.097400
1       Lasso  12.097939
2  ElasticNet  19.497338


In [4]:
# 필요한 라이브러리 임포트
import pandas as pd  # 데이터 처리를 위한 pandas
import numpy as np  # 수치 계산을 위한 numpy
from sklearn.model_selection import train_test_split, cross_val_score  # 데이터 분할 및 교차 검증
from sklearn.impute import SimpleImputer  # 결측치 처리
from sklearn.preprocessing import OneHotEncoder, StandardScaler  # 범주형 변수 인코딩 및 수치형 변수 스케일링
from sklearn.compose import ColumnTransformer  # 컬럼별 전처리 파이프라인 구성
from sklearn.pipeline import Pipeline  # 전체 전처리 및 모델링 파이프라인 구성
from sklearn.linear_model import Ridge, Lasso, ElasticNet  # 선형 회귀 모델
from sklearn.metrics import mean_squared_error, make_scorer  # 모델 평가 지표
from sklearn.model_selection import RandomizedSearchCV  # 랜덤 서치를 통한 하이퍼파라미터 튜닝
from skopt import BayesSearchCV  # 베이지안 최적화를 통한 하이퍼파라미터 튜닝
import xgboost as xgb  # XGBoost 모델
import lightgbm as lgb  # LightGBM 모델
import warnings  # 경고 메시지 처리
import joblib  # 모델 저장 및 로드
import os  # 파일 시스템 작업
import json  # JSON 파일 처리
from datetime import datetime  # 날짜/시간 처리를 위한 datetime

# LightGBM의 불필요한 경고 메시지 무시 설정
warnings.filterwarnings('ignore', category=UserWarning, module='lightgbm')

# 데이터 로드
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

# 특성 선택
drop_cols = ['id', 'Calories']
numeric_feats = train.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_feats = train.select_dtypes(include=['object']).columns.tolist()

# drop_cols 제외
numeric_feats = [col for col in numeric_feats if col not in drop_cols]
categorical_feats = [col for col in categorical_feats if col not in drop_cols]
main_features = numeric_feats + categorical_feats

print("Numeric features:", numeric_feats)
print("Categorical features:", categorical_feats)
print("Total features:", len(main_features))

# 훈련/검증 데이터 분할
y = train['Calories']
X = train[main_features]
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"\n훈련 데이터 크기: {X_train.shape}")
print(f"검증 데이터 크기: {X_val.shape}")

# 수치형 변수 전처리 파이프라인 구성
numeric_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),  # 결측치 처리
    ('scaler', StandardScaler())  # 표준화 스케일링 적용
])

# 범주형 변수 전처리 파이프라인 구성
categorical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),  # 결측치 처리
    ('encoder', OneHotEncoder(handle_unknown='ignore'))  # 원-핫 인코딩 적용
])

# 전처리 파이프라인 통합
preprocessor = ColumnTransformer([
    ('num', numeric_transformer, numeric_feats),  # 수치형 데이터 처리
    ('cat', categorical_transformer, categorical_feats),  # 범주형 데이터 처리
])

# 2. 모델 후보군 정의
models = {
    'Ridge': Ridge(),  # 릿지 회귀
    'Lasso': Lasso(),  # 라쏘 회귀
    'ElasticNet': ElasticNet(),  # 엘라스틱넷 회귀
    'XGBoost': xgb.XGBRegressor(tree_method='hist', random_state=42),  # XGBoost
    'LightGBM': lgb.LGBMRegressor(random_state=42, verbose=-1)  # LightGBM
}

results = {}  # 결과 저장 딕셔너리
best_model = None
best_rmse = float('inf')

for name, model in models.items():
    print(f'\n==== {name} ====')
    try:
        # 3.1 전체 파이프라인 구성 (전처리 + 모델)
        pipe = Pipeline([
            ('preprocessor', preprocessor),  # 전처리 단계
            ('reg', model)  # 모델 단계
        ])
        
        # 3.2 모델 학습
        pipe.fit(X_train, y_train)
        
        # 3.3 검증 데이터로 예측 및 성능 평가
        y_val_pred = pipe.predict(X_val)
        rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))
        print(f'Validation RMSE: {rmse:.4f}')
        
        results[name] = rmse  # 결과 저장
        
        # 최고 성능 모델 저장
        if rmse < best_rmse:
            best_rmse = rmse
            best_model = pipe
            
    except Exception as e:
        print(f"Error: {str(e)}")

# 4. 최종 결과 비교 및 정렬
print("\n==== Final Results ====")
results_df = pd.DataFrame(results.items(), columns=['Model', 'RMSE'])
results_df = results_df.sort_values('RMSE')
print(results_df)

# 5. 최고 성능 모델로 테스트 데이터 예측
print("\n==== Test Data Prediction ====")
X_test = test[main_features]
test_predictions = best_model.predict(X_test)

# 6. Submission 파일 생성
submission = pd.DataFrame({
    'id': test['id'],
    'Calories': test_predictions
})

# 현재 시간을 파일명에 포함
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
submission_filename = f'submission_{timestamp}.csv'
submission.to_csv(submission_filename, index=False)
print(f"\nSubmission 파일이 생성되었습니다: {submission_filename}")

# 7. 최고 성능 모델 저장
model_filename = f'best_model_{timestamp}.joblib'
joblib.dump(best_model, model_filename)
print(f"최고 성능 모델이 저장되었습니다: {model_filename}")

Numeric features: ['Age', 'Height', 'Weight', 'Duration', 'Heart_Rate', 'Body_Temp']
Categorical features: ['Sex']
Total features: 7

훈련 데이터 크기: (600000, 7)
검증 데이터 크기: (150000, 7)

==== Ridge ====
Validation RMSE: 11.0588

==== Lasso ====
Validation RMSE: 12.0410

==== ElasticNet ====
Validation RMSE: 19.4038

==== XGBoost ====
Validation RMSE: 3.8012

==== LightGBM ====


C:\Users\Admin\Desktop\lgu6P_ml_dl\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
C:\Users\Admin\Desktop\lgu6P_ml_dl\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


Validation RMSE: 3.7966

==== Final Results ====
        Model       RMSE
4    LightGBM   3.796576
3     XGBoost   3.801155
0       Ridge  11.058772
1       Lasso  12.040954
2  ElasticNet  19.403807

==== Test Data Prediction ====

Submission 파일이 생성되었습니다: submission_20250529_113233.csv
최고 성능 모델이 저장되었습니다: best_model_20250529_113233.joblib
